In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm, binom, bernoulli
import matplotlib.pyplot as plt

import statsmodels.formula.api as smf

import itertools
import random
import math

/home/warrior/miniconda3/envs/brain/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [44]:
N = 4
DENSITY = 0.4
NOISE_LEVEL = 0.3

In [45]:
r = 2 #EDGE SIZE

E = math.factorial(N) / (math.factorial(r)*math.factorial(N- r))
print(f'EDGES (Combinations): {E:.0f}\n')

E = N*(N-1) / 2
print(f'EDGES (Hand-shake): {E:.0f}')

EDGES (Combinations): 6

EDGES (Hand-shake): 6


In [46]:
nodes = np.arange(N)
nodes

array([0, 1, 2, 3])

In [47]:
edges = []
combinations = itertools.combinations(range(N), r)
for idx , e in enumerate(combinations):
    p = np.random.rand()
    #print(idx, p, e)
    if p <= DENSITY:
        edges.append(e)

In [48]:
print(len(edges))
edges

2


[(0, 1), (1, 3)]

In [49]:
coefficients = [np.random.rand() for e in edges]
coefficients

[0.9672577534982904, 0.26438710245328667]

In [50]:
coeff_dict = {}
parent_dict = {}
for idx, e in enumerate(edges):
    u, v = e[0], e[1]
    coeff_dict[v] = coefficients[idx]
    parent_dict[v] = u
print(coeff_dict,'\n')
print(parent_dict)

{1: 0.9672577534982904, 3: 0.26438710245328667} 

{1: 0, 3: 1}


In [51]:
parents = list(parent_dict.values())
parents

[0, 1]

In [52]:
T = 1000
system = {}
for p in parents:
    xs = norm.rvs(size=T)
    system[p] = xs
system.keys()

dict_keys([0, 1])

In [53]:
coefficients = [np.random.rand() for e in edges]
coefficients

[0.29411135002382605, 0.6230113916769325]

In [54]:
coeff_dict = {}
for idx, e in enumerate(edges):
    u, v = e[0], e[1]
    coeff_dict[v] = coefficients[idx]
    print(f'{v} = {coeff_dict[v]:.2f} * {u}')

1 = 0.29 * 0
3 = 0.62 * 1


In [55]:
T = 1000
system = {}
for p in parents:
    xs = norm.rvs(size=T)
    system[p] = xs
system.keys()

dict_keys([0, 1])

In [56]:
coeff_dict = {}
parent_dict = {}
for idx, e in enumerate(edges):
    u, v = e[0], e[1]
    coeff_dict[v] = coefficients[idx]
    parent_dict[v] = u
print(coeff_dict,'\n')
print(parent_dict)

{1: 0.29411135002382605, 3: 0.6230113916769325} 

{1: 0, 3: 1}


In [57]:
for idx, e in enumerate(edges):
    u, v = e[0], e[1]
    #print(u,v)
    c = coeff_dict[v]
    #print(idx, p, e)
    sign = 1
    p = np.random.rand()
    if p > 0.8: # 
        sign = -1
    
    system[v] = sign*c*system[u] + NOISE_LEVEL*norm.rvs(size=T) # DYNAMICS

    print(f'({v}<-{u}):  X{str(v)} <- {c:.2f}*X{str(u)} + err')
    print(f'({v}<-{u}):  X{str(v)} <- {sign*c:.2f}*X{str(u)} + err')
    print('')

(1<-0):  X1 <- 0.29*X0 + err
(1<-0):  X1 <- -0.29*X0 + err

(3<-1):  X3 <- 0.62*X1 + err
(3<-1):  X3 <- 0.62*X1 + err



In [58]:
pd.DataFrame(system).head()

,0,1,3
0,0.785558,0.043676,0.190072
1,-0.484350,0.217131,0.486008
2,-0.184296,0.172677,0.018444
3,-0.032991,-0.325348,-0.225621
4,0.972167,-0.543642,-0.210886


In [59]:
kernel = system.keys()
kernel

dict_keys([0, 1, 3])

In [60]:
shell = [n for n in nodes if n not in kernel]
shell

[2]

In [61]:
for n in shell:
    system[n] = norm.rvs(size=T)

In [62]:
system_df = pd.DataFrame(system)
system_df.head()

,0,1,3,2
0,0.785558,0.043676,0.190072,0.399793
1,-0.484350,0.217131,0.486008,0.754017
2,-0.184296,0.172677,0.018444,-0.358288
3,-0.032991,-0.325348,-0.225621,0.505183
4,0.972167,-0.543642,-0.210886,-2.148327


In [63]:
system_df = system_df[nodes]
system_df.head()

,0,1,2,3
0,0.785558,0.043676,0.399793,0.190072
1,-0.484350,0.217131,0.754017,0.486008
2,-0.184296,0.172677,-0.358288,0.018444
3,-0.032991,-0.325348,0.505183,-0.225621
4,0.972167,-0.543642,-2.148327,-0.210886


In [64]:
system_df.to_csv('data/system.csv', index=False)